<a href="https://colab.research.google.com/github/mayfluf/projeto-tainah-elt/blob/main/Projeto_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ETL COMPLETO - Projeto Mercado Livre

# Bibliotecas
import pandas as pd
import requests
import mysql.connector
from pymongo import MongoClient
from google.cloud import storage, bigquery
from google.colab import auth
from mysql.connector import Error


In [ ]:
# 1 - Coleta de dados via API do Mercado Livre
termo_de_busca = "nootbook"
url = f"https://api.mercadolibre.com/sites/MLB/search?q={termo_de_busca}"
response = requests.get(url)

if response.status_code == 200:
    dados = response.json()
    produtos = [
        {
            "produto": item.get('title'),
            "valor": item.get('price'),
            "link": item.get('permalink'),
            "condicao": item.get('condition'),
            "categoria": item.get('category_id')
        } for item in dados.get('results', [])
    ]
    df = pd.DataFrame(produtos)
else:
    raise Exception(f"Erro ao acessar API: {response.status_code}")


In [ ]:
# 2 - Transformações
# Já foram aplicadas durante a extração: nomes padronizados, colunas relevantes, remoção de nulos se necessário
df.dropna(inplace=Tr

In [ ]:
# 3 - Salvando CSV
csv_path = 'mercado.csv'
df.to_csv(csv_path, index=False)



In [ ]:
# 4 - Upload para bucket GCP
auth.authenticate_user()
project_id = "turmas-dados"
!gcloud config set project {project_id}

bucket_name = "etl-basico"
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob("mercadolivre_douglas/mercado.csv")
blob.upload_from_filename(csv_path)
print(f"Arquivo CSV enviado ao bucket: {bucket_name}")

In [ ]:

# 5 - Envio para MongoDB Atlas
mongo_uri = "mongodb+srv://<usuario>:<senha>@cluster0.xu3hl.mongodb.net/?retryWrites=true&w=majority"
mongo_client = MongoClient(mongo_uri)
mongo_db = mongo_client['mercado']
mongo_collection = df.to_dict(orient='records')
mongo_db.webcam.insert_many(mongo_collection)


In [ ]:
# 6 - Inserção em Cloud SQL (MySQL)
config = {
    'user': 'root',
    'password': '',
    'host': '34.70.26.188',
    'database': 'mercado'
}

try:
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS webcam (
            produto VARCHAR(255),
            valor DECIMAL(11,2),
            link VARCHAR(255),
            condicao VARCHAR(255),
            categoria VARCHAR(255)
        );
    """)
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO webcam (produto, valor, link, condicao, categoria)
            VALUES (%s, %s, %s, %s, %s);
        """, tuple(row))
    conn.commit()
    print("Dados enviados ao MySQL com sucesso!")
except Error as e:
    print("Erro ao conectar ao MySQL:", e)
finally:
    if conn.is_connected():
        cursor.close()
        conn.close()



In [ ]:

# 7 - Envio para BigQuery (mercado2)
client_bq = bigquery.Client()
table_id = 'turmas-dados.mercado2.webcam'
df.to_gbq(table_id, project_id=project_id, if_exists='replace')
